<a href="https://colab.research.google.com/github/devinbook/Tagalog-profanity-detection/blob/main/Bert_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, XLNetTokenizer, BertForSequenceClassification, XLNetForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.special import softmax
import torch

In [2]:
df = pd.read_csv('/content/tagalog_profanity_feature_extracted (1).csv')

In [3]:
# Tokenization for both BERT and XLNet
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [4]:
# Function to tokenize data with improved handling
def tokenize_data(texts, tokenizer):
    return tokenizer(
        [str(text) for text in texts],  # Convert each text entry to a string
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=256  # Set max length to 512 for compatibility with most transformer models
    )

In [5]:
# Tokenize the text for both BERT and XLNet
tokenized_bert = tokenize_data(df['Text'].tolist(), bert_tokenizer)
tokenized_xlnet = tokenize_data(df['Text'].tolist(), xlnet_tokenizer)

# Step 4: Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['Text'], df['Label'], test_size=0.2)

# Tokenize the split data
train_bert = tokenize_data(train_texts.tolist(), bert_tokenizer)
test_bert = tokenize_data(test_texts.tolist(), bert_tokenizer)
train_xlnet = tokenize_data(train_texts.tolist(), xlnet_tokenizer)
test_xlnet = tokenize_data(test_texts.tolist(), xlnet_tokenizer)

# Convert labels to torch tensors
train_labels = torch.tensor(train_labels.values)
test_labels = torch.tensor(test_labels.values)

In [6]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
xlnet_model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments

# Define a custom function to calculate metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)  # Get the predicted class
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Set up TrainingArguments with evaluation per epoch
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,               # Adjust as needed
    per_device_train_batch_size=16,   # Adjust as needed
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    fp16=True,                        # Enable mixed precision training for faster processing
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from datasets import Dataset
import torch

# Convert train and test data to Hugging Face Dataset format
train_data = Dataset.from_dict({
    'input_ids': train_bert['input_ids'],
    'attention_mask': train_bert['attention_mask'],
    'labels': train_labels
})

test_data = Dataset.from_dict({
    'input_ids': test_bert['input_ids'],
    'attention_mask': test_bert['attention_mask'],
    'labels': test_labels
})

# Set up Trainer with the modified train and test datasets
trainer_bert = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [11]:
trainer_bert.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.117800,0.107296,0.976349,0.976376,0.976349,0.976348
2,0.061900,0.064858,0.986815,0.986818,0.986815,0.986815
3,0.046300,0.057152,0.988446,0.988490,0.988446,0.988446
4,0.027500,0.075593,0.987631,0.987705,0.987631,0.987631
5,0.018800,0.055492,0.990077,0.990133,0.990077,0.990078
6,0.010600,0.068281,0.988582,0.988697,0.988582,0.988582
7,0.005700,0.058111,0.991845,0.991866,0.991845,0.991845
8,0.004100,0.069116,0.991165,0.991196,0.991165,0.991165
9,0.001100,0.070370,0.991845,0.991852,0.991845,0.991845
10,0.000400,0.074125,0.992116,0.992120,0.992116,0.992116


TrainOutput(global_step=18400, training_loss=0.035210287159956666, metrics={'train_runtime': 4864.819, 'train_samples_per_second': 60.489, 'train_steps_per_second': 3.782, 'total_flos': 3.87128451303936e+16, 'train_loss': 0.035210287159956666, 'epoch': 10.0})

In [13]:
test_encodings = tokenize_data(test_texts.tolist(), bert_tokenizer)
test_labels_tensor = torch.tensor(test_labels.tolist(), dtype=torch.long)

In [14]:
from torch.utils.data import Dataset, DataLoader

class ProfanityTestDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = ProfanityTestDataset(test_encodings, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [16]:
model = bert_model

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()  # Set model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        # Move batch data to the appropriate device
        inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(model.device)

        # Get model predictions
        outputs = model(**inputs)
        logits = outputs.logits

        # Convert logits to class predictions
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

<ipython-input-14-6ca7f284852e>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-14-6ca7f284852e>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Test Accuracy: 0.9921
Test Precision: 0.9921
Test Recall: 0.9921
Test F1 Score: 0.9921


In [19]:
from transformers import BertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the model (use your trained model here)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Save both model and tokenizer
model.save_pretrained('./model')  # Save model
tokenizer.save_pretrained('./tokenizer')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json')

In [20]:
# Save model and tokenizer to Google Drive folder
model.save_pretrained("/content/model")
tokenizer.save_pretrained("/content/tokenizer")

('/content/tokenizer/tokenizer_config.json',
 '/content/tokenizer/special_tokens_map.json',
 '/content/tokenizer/vocab.txt',
 '/content/tokenizer/added_tokens.json')

In [21]:
model_path = "/content/drive/MyDrive/MyModels/model"
tokenizer_path = "/content/drive/MyDrive/MyModels/tokenizer"

# Save model and tokenizer to Google Drive
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/MyModels/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/MyModels/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/MyModels/tokenizer/vocab.txt',
 '/content/drive/MyDrive/MyModels/tokenizer/added_tokens.json')